In [132]:
# Name: Abdullahi Sani Shuaibu
# ID: 202303370
# Project: Embedded Markov Proces based Model for Performance Analysis of IDPS 
# Course: COE 520 (Queuing Theory and Network Applications)
# Instructor: Dr. Mohammad Abdurazzak Felemban
# Term: 231
# Date: 6 October, 2023

### Constants

In [149]:
L = 3 # L-1 is queue length
L_list = [5, 10, 25, 50, 100]
p = 0.3 # probability of early decision
p_list = [0.25, 0.5, 0.75, 0.9]
r = 1 - p # probability of content checking
lams = [lam for lam in range(0, 500, 25)]

### Settings

In [134]:
micro = 10**-6;
settings  = {
    1: {'alpha': (1/0.5)*micro, 'mu': (1/4)*micro},
    2: {'alpha': (1/0.5)*micro, 'mu': (1/8)*micro},
    3: {'alpha': (1/0.5)*micro, 'mu': (1/12)*micro},
    4: {'alpha': (1/5.0)*micro, 'mu': (1/12)*micro},
    }

### Coefficients

In [135]:
w01 = w02 = 1
w11 = lambda setting, lam: ((setting['alpha'] + lam)*lam)/((setting['alpha'] + p*lam)*setting['mu'])
w12 = lambda setting, lam: (lam - p*lam)/(setting['alpha'] + p*lam)

def wn1(n, setting, lam):
    """ for 2 <= n < L """
    if n == 0:
        return w01
    if n == 1:
        return w11(setting, lam)
    return (((setting['alpha'] + lam)*(lam + setting['mu']))/((p*lam + setting['alpha'])*setting['mu']))*wn1(n-1, setting, lam) - ((lam + setting['alpha'])/(lam*p + setting['alpha']))*wn1(n-2, setting, lam) - ((lam*setting['alpha'])/((lam*p + setting['alpha'])*setting['mu']))*wn2(n-1, setting, lam)

def wn2(n, setting, lam):
    """ for 2 <= n < L """
    if n == 0:
        return w02
    if n == 1:
        return w12(setting, lam)
    return (lam/(lam + setting['alpha']))*wn2(n-1, setting, lam) + (((1 - p)*setting['mu'])/(lam + setting['alpha']))*wn1(n, setting, lam)

# print(wn1(15, settings[1], 5))
# print(wn2(15, settings[1], 5))

### Probabilities

In [136]:
q00 = lambda setting, lam, L=L: 1/(1 + sum([wn1(n, setting, lam) + wn2(n, setting, lam) for n in range(1, L+1)]))

def qn1(n, setting, lam, L=L, p=p): # p not needed
    if n == L and L > 1:
        return ((lam)/(setting['mu']))*wn1(L-1, setting, lam)*q00(setting, lam, L)
    if n == L and L == 1:
        return ((lam)/(setting['mu']))*q00(setting, lam, L)
    return wn1(n, setting, lam)*q00(setting, lam, L)
    
def qn2(n, setting, lam, L=L, p=p):
    if n == L and L > 1:
        return ((lam)/(setting['alpha']))*(wn2(L-1, setting, lam) + (1-p)*wn1(L-1, setting, lam))*q00(setting, lam, L)
    if n == L and L == 1:
        return ((1-p)*lam)/(setting['alpha'])*q00(setting, lam, L)
    return wn2(n, setting, lam)*q00(setting, lam, L)
    
# print('q00: ', q00(settings[2], 5))
# print('q21: ', qn1(2, settings[2], 5))
# print('q22: ', qn2(2, settings[2], 5))

## Performance Metrices

#### Throughput

In [137]:
def throughput(setting, lam, L=L, p=p):
    return p*setting['mu']*sum([qn1(n, setting, lam, L, p) for n in range(1, L+1)]) + setting['alpha']*sum([qn2(n, setting, lam, L, p) for n in range(1, L+1)])

#### Probability of Packet Loss

In [138]:
def ppl(setting, lam, L=L, p=p):
    return qn1(L, setting, lam, L, p) + qn2(L, setting, lam, L, p)

#### Average Number of Packets in the System

In [139]:
def avgn(setting, lam, L=L, p=p):
    return sum([n*(qn1(n, setting, lam, L, p) + qn2(n, setting, lam, L, p)) for n in range(1, L+1)])

#### Average Packet Time in the System

In [140]:
def avgt(setting, lam, L=L, p=p):
    return avgn(setting, lam, L, p)/throughput(setting, lam, L, p)

#### Average Service Time of the Two Stages

In [141]:
def avgst(setting):
    return (1/setting['mu']) + ((1-p)/setting['alpha'])

#### Average Packet Queue Time

In [142]:
def avgqt(setting, lam, L=L, p=p):
    return avgt(setting, lam, L, p) - avgst(setting)

## Generate Data

In [ ]:
import csv
import os
import json

# for k, v in settings.items():
#     print(k, v)
def data_gen(type='settings'):
    data_dir = os.path.join('data', type)
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)
    
    csv_path = os.path.join(data_dir, type + '.csv')
    json_path = os.path.join(data_dir, type + '.json')
    csv_file= open(csv_path, mode='w')
    json_file= open(json_path, mode='w')
        
    csv_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    header = [str(lam) for lam in lams]

    if type == 'settings':
        data = {}
        for setting in settings:
            data[setting] = {}
        for setting in settings:
            for lam in lams:
                this_point = {}
                this_point['throughput'] = throughput(setting=settings[setting], lam=lam)
                this_point['ppl'] = ppl(setting=settings[setting], lam=lam)
                this_point['delay'] = avgt(setting=settings[setting], lam=lam)
                data[setting].update({lam: this_point})

            csv_writer.writerow([f'setting: {setting}'])
            csv_writer.writerow(header)
            throughputs = []
            ppls = []
            delays = []
            for lam in data[setting]:
                throughputs.append(data[setting][lam]['throughput'])
                ppls.append(data[setting][lam]['ppl'])
                delays.append(data[setting][lam]['delay'])
            csv_writer.writerow(throughputs)
            csv_writer.writerow(ppls)
            csv_writer.writerow(delays)
        json.dump(data, json_file)

    if type == 'population':
        data = {}
        for population in L_list:
            data[population] = {}
        for population in L_list:
            for lam in lams:
                this_point = {}
                this_point['throughput'] = throughput(setting=settings[1], lam=lam, L=population)
                this_point['ppl'] = ppl(settings[1], lam, L=population)
                this_point['delay'] = avgt(settings[1], lam, L=population)
                data[population].update({lam: this_point})

            csv_writer.writerow([f'population: {population}'])
            csv_writer.writerow(header)
            throughputs = []
            ppls = []
            delays = []
            for lam in data[population]:
                throughputs.append(data[population][lam]['throughput'])
                ppls.append(data[population][lam]['ppl'])
                delays.append(data[population][lam]['delay'])
            csv_writer.writerow(throughputs)
            csv_writer.writerow(ppls)
            csv_writer.writerow(delays)
        json.dump(data, json_file)

    if type == 'probabilities':
        data = {}
        for prob in p_list:
            data[prob] = {}
        for prob in p_list:
            for lam in lams:
                this_point = {}
                this_point['throughput'] = throughput(setting=settings[1], lam=lam, p=prob)
                this_point['ppl'] = ppl(settings[1], lam, p=prob)
                this_point['delay'] = avgt(settings[1], lam, p=prob)
                data[prob].update({lam: this_point})

            csv_writer.writerow([f'probability: {prob}'])
            csv_writer.writerow(header)
            throughputs = []
            ppls = []
            delays = []
            for lam in data[prob]:
                throughputs.append(data[prob][lam]['throughput'])
                ppls.append(data[prob][lam]['ppl'])
                delays.append(data[prob][lam]['delay'])
            csv_writer.writerow(throughputs)
            csv_writer.writerow(ppls)
            csv_writer.writerow(delays)
        json.dump(data, json_file)

# for type in ['settings', 'probabilities', 'population']:
#     data_gen(type=type)

## Load Data

In [146]:
import json
import pandas as pd

# settings
data_json_settings = json.load(open('./data/settings/settings.json', 'r'))
data_csv_settings = pd.read_csv('./data/settings/settings.csv')

# probabilities
data_json_probabilities = json.load(open('./data/probabilities/probabilities.json', 'r'))
data_csv_probabilities = pd.read_csv('./data/probabilities/probabilities.csv')

# population
data_json_population = json.load(open('./data/population/population.json', 'r'))
data_csv_population = pd.read_csv('./data/population/population.csv')

## Extract Data

In [ ]:
import numpy as np

# lams = np.array(lams)

# # settings
# settings = {

# }

## Visualizations for Security Levels (L=25, p=0.3)

In [150]:
import matplotlib.pyplot as plt

### Throughput

In [152]:
# plt.plot(lams, np.array([]))

array({'a': 1, 'b': 2}, dtype=object)

### Packet Loss Ratio

### Average Packet Time in System

## Visualizations for System Population (Security Level 1, p=0.3)

### Throughput

### Packet Loss Ratio

### Average Packet Time in System